# Best Practice Audit Open Research.

This notebook provides an overview of the code to read in the data extracted from the best practice audit of model sharing. 

The dataset is a subset of the main review - limited to models shared.  The type of model shared is coded as **Visual Interactive Modelling (VIM)** based (e.g Anylogic, Simul8, Arena) versus **CODE** (e.g. Matlab, Python, SimPy, Java, R Simmer).

Additional fields have been extracted as part of the best practice review.

* `model_format` - VIM or CODE
* `model_has_doi`  - do the model artefacts have their own minted DOI? (0/1)
* `orcid` - do the researchers provide an ORCID with the model? (0/1)
* `license` - does the model have an explicit license defining how it can be used? (str)
* `readme` - is there an obvious file(s) where a user would look first? (0/1)             
* `link_to_paper` - does the model repository contain a link back to the pre-print or peer reviewed article? (0/1)
* `steps_run` - are there steps to run a model? (0/1)
* `formal_dep_mgt` - has the model been shared with formal software dependency management? (0/1)
* `informal_dep_mgt` - have any informal methods of dependency management been shared?  E.g. a list of software requirements. (0/1)
* `evidence_testing` - do the model and artefacts in the repository contain any evidence that they have been tested? (0/1)
* `downloadable` - can the model and artefacts be downloaded and executed locally? (0/1)
* `interactive_online` - can the model and its artefacts be executed online without local installation? (0/1)

## 1. Imports

### 1.1. Standard

In [1]:
import pandas as pd
import numpy as np

### 1.2 Preprocessing

In [2]:
from preprocessing import load_clean_bpa

## 2. Constants

In [22]:
FILE_NAME = 'https://raw.githubusercontent.com/TomMonks/' \
    + 'des_sharing_lit_review/main/data/bp_audit.zip'

LICENSE_LABEL = 'license'
NONE_LABEL = 'None'

In [4]:
## 3. Helper functions

def balance_of_model_format(df):
    unique_elements, counts_elements = np.unique(df['model_format'], 
                                                 return_counts=True)
    return unique_elements, counts_elements

## 3. Load and inspect dataset

The clean data set has 27 fields included.  These are listed below.  

In [5]:
clean = load_clean_bpa(FILE_NAME)
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   model_format                  39 non-null     category
 1   key                           39 non-null     object  
 2   item_type                     39 non-null     category
 3   pub_yr                        39 non-null     int64   
 4   author                        39 non-null     object  
 5   doi                           38 non-null     object  
 6   reporting_guidelines_mention  39 non-null     category
 7   covid                         39 non-null     category
 8   sim_software                  39 non-null     object  
 9   foss_sim                      39 non-null     category
 10  model_archive                 4 non-null      object  
 11  model_repo                    18 non-null     object  
 12  model_journal_supp            9 non-null      object

## 4. Results

## 4.1 What proportion of the share model artefacts has a DOI and guarantees on persistence?

In [6]:
unique_elements, counts_elements = np.unique(clean['model_has_doi'], 
                                                   return_counts=True)

has_doi = counts_elements[1]
has_doi_percent = (has_doi / len(clean)) * 100
result = f'A total of {has_doi} ({has_doi_percent:.1f}\%) models ' \
    + 'were provided with a DOI.'
print(result)

A total of 6 (15.4\%) models were provided with a DOI.


## 4.2 What proportion of artefacts are linked to the researcher via ORCID(s)?

In [7]:
unique_elements, counts_elements = np.unique(clean['orcid'], 
                                                   return_counts=True)

has_orcid = counts_elements[1]
has_orcid_percent = (has_orcid / len(clean)) * 100
result = f'A total of {has_orcid} ({has_orcid_percent:.1f}\%) models ' \
    + 'were provided were linked to a researcher via an ORCID.'
print(result)

A total of 6 (15.4\%) models were provided were linked to a researcher via an ORCID.


Of this small number what was the format of the model sharing.

In [8]:
orcids = clean[clean['orcid'] == 1]
model_format, counts = balance_of_model_format(orcids)
print(model_format, counts)

['CODE' 'VIM'] [3 3]


## 5. License

### 5.1 What proportion of models have an open license?

We extracted the type of license included with each shared model. When no license was included we recorded this as `None`. For one model shared as supplementary material with a journal we were unable to determine what license had been applied.  We labelled this as `Unknown`.  When a model was published as journal supplementary material we assigned the same license as applied to the paper if it was not explicitly stated.  For example, if a paper was published under a CC-BY 4.0 license and there was no explicit license attached to supplementary material we assumed the same license for the model.

> Note that models might be shared by a combination of approaches.  For example Zenodo + Github.

In [23]:
def license_versus_no_license(df):
    '''
    Returns a tuple containined the (number of licensed models, not licensed)
    contained with the dataset.
    
    Parameters:
    -----------
    df: pd.DataFrame
        A dataset to analyse.  Could be full dataset or a partial subset
        
    Returns:
    --------
    tuple (int, int)
    
    '''
    n_not_lincensed = len(df[df[LICENSE_LABEL]==NONE_LABEL])
    return len(df) - n_not_lincensed, n_not_lincensed
        

In [31]:
licensed, not_licensed = license_versus_no_license(clean)
per_licensed, per_not_licensed = (licensed / len(clean)) * 100, (not_licensed / len(clean)) * 100

In [32]:
license_txt = f'Of the models shared a total of {licensed} ({per_licensed:.1f}\%)' \
    + 'had an open license attached.' 

print(license_txt)

Of the models shared a total of 18 (46.2\%)had an open license attached.


### 5.2 When a model is licensed what was the most popular license?

In [45]:
licenses, n_license = np.unique(clean[LICENSE_LABEL], 
                               return_counts=True)

license_results = pd.concat([pd.Series(licenses), pd.Series(n_license)], axis=1)
license_results.columns = ['License', 'n']     
license_results = license_results.set_index('License')
# drop none from the results
license_results = license_results.drop(NONE_LABEL)
license_results.sort_values(by='n', ascending=False)

,n
License,
CC-BY 4.0,6
GPL-3,5
MIT,2
Apache,1
CC BY-NC-ND 4.0,1
CC BY-NC-SA 4.0,1
CC-BY-NC 4.0,1
Unknown,1


In [15]:
def licences_by_sharing_tools(df):
    '''
    Return a DataFrame containing licenses (rows) by type of sharing
    i.e. archive, cloud repo, journal supp , personal/org website, platform.
    
    Parameters:
    -----------
    df: pd.DataFrame
        Contains data to analysis.  Eg.full dataset or subset
        
    Returns:
    -------
    DataFrame (9, 6)
    '''
    selected_columns = ['model_archive', 'model_repo', 'model_journal_supp',
                        'model_personal_org', 'model_platform']
    license_by_sharing = df.groupby(by='license')[selected_columns].count()
    #license_by_sharing['Total'] = license_by_sharing.sum(axis=1)
    #sorted_cols = ['Total'] + selected_columns
    return license_by_sharing.sort_values(by='model_repo', 
                                          ascending=False)

In [16]:
def format_license_table(df):
    '''
    Format the license table.
    '''
    column_headers = ['Archive', 'Repository', 
                      'Journal', 'Personal/org', 'Platform']
    df.columns = column_headers
    return df
    

In [17]:
format_license_table(licences_by_sharing_tools(clean))

,Archive,Repository,Journal,Personal/org,Platform
license,,,,,
None,0,10,3,5,6
GPL-3,0,5,0,0,0
MIT,1,2,0,0,1
Apache,0,1,0,0,0
CC BY-NC-ND 4.0,0,0,1,0,0
CC BY-NC-SA 4.0,0,0,0,0,1
CC-BY 4.0,3,0,3,0,0
CC-BY-NC 4.0,0,0,1,0,0
Unknown,0,0,1,0,0


The table shows that the majority of the models shared via a cloud hosted code repository (e.g. github) do not have an open license.  This means the author has exclusive copyright and the code cannot legally be reused by anyone.  There are some minor exceptions; for example, code on GitHubs can be viewed and forked, but not edited or used. The majority of the remaining models also have no license (one in journal supplementary material is marked as an unknown as its unclear from the journal if it is open or not) including six that sit in a cloud platform.  CC-BY type licenses are supplied with eight models.  The Turing way considers these to be less suitable for data science artefacts such as code and models and recommends licenses such as GPL-3 or MIT.  One model was shared with a CC BY-NC-ND 4.0 i.e no derivative works.